In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
from sklearn.metrics import accuracy_score
from mlxtend.plotting import plot_decision_regions
import tensorflow
import keras
from keras.layers import Dense
from keras.models import Sequential
import optuna
import xgboost as xgb
import catboost

In [26]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv')

In [28]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3116945 entries, 0 to 3116944
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   class                 object 
 2   cap-diameter          float64
 3   cap-shape             object 
 4   cap-surface           object 
 5   cap-color             object 
 6   does-bruise-or-bleed  object 
 7   gill-attachment       object 
 8   gill-spacing          object 
 9   gill-color            object 
 10  stem-height           float64
 11  stem-width            float64
 12  stem-root             object 
 13  stem-surface          object 
 14  stem-color            object 
 15  veil-type             object 
 16  veil-color            object 
 17  has-ring              object 
 18  ring-type             object 
 19  spore-print-color     object 
 20  habitat               object 
 21  season                object 
dtypes: float64(3), int64(1), object(18)
memory

In [29]:
train_df

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,e,9.29,f,NaN,n,t,NaN,NaN,w,...,b,NaN,w,u,w,t,g,NaN,d,u
3116941,3116941,e,10.88,s,NaN,w,t,d,c,p,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
3116942,3116942,p,7.82,x,e,e,f,a,NaN,w,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
3116943,3116943,e,9.45,p,i,n,t,e,NaN,p,...,NaN,y,w,NaN,NaN,t,p,NaN,d,u


In [30]:
train_df.isnull().sum()

id                            0
class                         0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

In [31]:
train_new = train_df.drop(['stem-root','stem-surface','veil-type','veil-color','spore-print-color'],axis='columns')

In [32]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3116945 entries, 0 to 3116944
Data columns (total 17 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   class                 object 
 2   cap-diameter          float64
 3   cap-shape             object 
 4   cap-surface           object 
 5   cap-color             object 
 6   does-bruise-or-bleed  object 
 7   gill-attachment       object 
 8   gill-spacing          object 
 9   gill-color            object 
 10  stem-height           float64
 11  stem-width            float64
 12  stem-color            object 
 13  has-ring              object 
 14  ring-type             object 
 15  habitat               object 
 16  season                object 
dtypes: float64(3), int64(1), object(13)
memory usage: 404.3+ MB


In [33]:
#fill null values
train_new['cap-diameter'] = train_new['cap-diameter'].fillna(train_new['cap-diameter'].median())
train_new['stem-height'] = train_new['stem-height'].fillna(train_new['stem-height'].median())
train_new['stem-width'] = train_new['stem-width'].fillna(train_new['stem-width'].median())

In [34]:
train_new['cap-shape'] = train_new['cap-shape'].fillna(train_new['cap-shape'].value_counts().index[0])
train_new['cap-surface'] = train_new['cap-surface'].fillna(train_new['cap-surface'].value_counts().index[0])
train_new['cap-color'] = train_new['cap-color'].fillna(train_new['cap-color'].value_counts().index[0])
train_new['does-bruise-or-bleed'] = train_new['does-bruise-or-bleed'].fillna(train_new['does-bruise-or-bleed'].value_counts().index[0])
train_new['gill-attachment'] = train_new['gill-attachment'].fillna(train_new['gill-attachment'].value_counts().index[0])
train_new['gill-spacing'] = train_new['gill-spacing'].fillna(train_new['gill-spacing'].value_counts().index[0])
train_new['gill-color'] = train_new['gill-color'].fillna(train_new['gill-color'].value_counts().index[0])
train_new['stem-color'] = train_new['stem-color'].fillna(train_new['stem-color'].value_counts().index[0])
train_new['has-ring'] = train_new['has-ring'].fillna(train_new['has-ring'].value_counts().index[0])
train_new['ring-type'] = train_new['ring-type'].fillna(train_new['ring-type'].value_counts().index[0])
train_new['habitat'] = train_new['habitat'].fillna(train_new['habitat'].value_counts().index[0])
train_new['season'] = train_new['season'].fillna(train_new['season'].value_counts().index[0])

In [35]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_new['cap-shape'] = le.fit_transform(train_new['cap-shape'])
train_new['cap-surface'] = le.fit_transform(train_new['cap-surface'])
train_new['cap-color'] = le.fit_transform(train_new['cap-color'])
train_new['does-bruise-or-bleed'] = le.fit_transform(train_new['does-bruise-or-bleed'])
train_new['gill-attachment'] = le.fit_transform(train_new['gill-attachment'])
train_new['gill-spacing'] = le.fit_transform(train_new['gill-spacing'])
train_new['gill-color'] = le.fit_transform(train_new['gill-color'])
train_new['stem-color'] = le.fit_transform(train_new['stem-color'])
train_new['has-ring'] = le.fit_transform(train_new['has-ring'])
train_new['ring-type'] = le.fit_transform(train_new['ring-type'])
train_new['habitat'] = le.fit_transform(train_new['habitat'])
train_new['season'] = le.fit_transform(train_new['season'])

In [36]:
train_new['class'] = le.fit_transform(train_new['class'])

In [37]:
train_new

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
0,0,0,8.80,53,72,72,8,44,28,59,4.51,15.39,55,5,18,25,0
1,1,1,4.51,71,56,64,8,44,28,46,4.79,6.48,47,18,39,25,3
2,2,0,6.94,53,72,49,8,75,28,59,6.85,9.93,46,5,18,36,3
3,3,0,3.88,53,81,57,8,70,28,37,4.16,6.53,55,5,18,25,2
4,4,0,5.85,71,65,74,8,47,28,59,3.37,8.36,55,5,18,29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,0,9.29,53,76,63,20,44,28,59,12.14,18.81,55,18,19,25,2
3116941,3116941,0,10.88,67,76,74,20,47,28,48,6.65,26.97,55,5,18,25,2
3116942,3116942,1,7.82,71,53,55,8,44,28,59,9.51,11.06,57,18,39,25,0
3116943,3116943,0,9.45,64,59,63,20,52,28,48,9.13,17.77,55,18,27,25,2


In [38]:
x = train_new.iloc[:,2:]
y = train_new['class']

In [48]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x = pd.DataFrame(ss.fit_transform(x),columns=x.columns)

In [49]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [50]:
x_train.shape

(2181861, 15)

In [51]:
from keras.callbacks import EarlyStopping
from keras.regularizers import L2

In [52]:
ann = Sequential()

In [54]:
ann.add(Dense(12,input_dim= 15,activation='relu'))
ann.add(Dense(10,activation='relu'))
ann.add(Dense(8,activation='relu'))
ann.add(Dense(6,activation='relu'))
ann.add(Dense(4,activation='relu'))
ann.add(Dense(2,activation='relu'))
ann.add(Dense(1,activation='sigmoid'))

In [55]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [56]:
ann.fit(x_train,y_train,batch_size=1000,epochs=50,validation_data=(x_test,y_test),callbacks=EarlyStopping())

Epoch 1/50
2182/2182 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8019 - loss: 0.4422 - val_accuracy: 0.9263 - val_loss: 0.2373
Epoch 2/50
2182/2182 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9317 - loss: 0.2223 - val_accuracy: 0.9412 - val_loss: 0.1975
Epoch 3/50
2182/2182 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9434 - loss: 0.1918 - val_accuracy: 0.9483 - val_loss: 0.1795
Epoch 4/50
2182/2182 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9523 - loss: 0.1679 - val_accuracy: 0.9584 - val_loss: 0.1513
Epoch 5/50
2182/2182 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9593 - loss: 0.1490 - val_accuracy: 0.9603 - val_loss: 0.1470
Epoch 6/50
2182/2182 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9611 - loss: 0.1439 - val_accuracy: 0.9611 - val_loss: 0.1439
Epoch 7/50
2182/2182 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9622 - loss: 0.1410 - val_accuracy: 0.9626 - val_loss: 0.1410
Epoch 8/50
2182/2182 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9632 - loss: 0.1383 - 

In [57]:
y_pred1=ann.predict(x_train)
prd_data1 = []
for i in y_pred1:
    if i[0] > 0.5:
        prd_data1.append(1)
    else:
        prd_data1.append(0)

68184/68184 ━━━━━━━━━━━━━━━━━━━━ 71s 1ms/step


In [58]:
y_pred=ann.predict(x_test)
prd_data = []
for i in y_pred:
    if i[0] > 0.5:
        prd_data.append(1)
    else:
        prd_data.append(0)

29222/29222 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step


In [59]:
accuracy_score(y_test,prd_data)*100

96.75718972840943

In [60]:
accuracy_score(y_train,prd_data1)*100

96.77999652590151

In [61]:
x.shape

(3116945, 15)

In [62]:
ann.fit(x,y,batch_size=1000,epochs=50,validation_data=(x_test,y_test),callbacks=EarlyStopping())

Epoch 1/50
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9683 - loss: 0.1218 - val_accuracy: 0.9700 - val_loss: 0.1172
Epoch 2/50
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9704 - loss: 0.1164 - val_accuracy: 0.9718 - val_loss: 0.1123
Epoch 3/50
3117/3117 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9711 - loss: 0.1136 - val_accuracy: 0.9717 - val_loss: 0.1130


In [ ]:
#TEST Data

In [63]:
test_df = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')

In [64]:
test_df

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,...,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,...,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,5194904,0.88,x,g,w,f,a,d,w,2.67,...,NaN,NaN,e,NaN,NaN,f,f,NaN,d,u
2077960,5194905,3.12,x,s,w,f,d,c,w,2.69,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
2077961,5194906,5.73,x,e,e,f,a,NaN,w,6.16,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
2077962,5194907,5.03,b,g,n,f,a,d,g,6.00,...,NaN,s,g,NaN,NaN,f,f,NaN,d,a


In [65]:
test_df.isnull().sum()

id                            0
cap-diameter                  7
cap-shape                    31
cap-surface              446904
cap-color                    13
does-bruise-or-bleed         10
gill-attachment          349821
gill-spacing             839595
gill-color                   49
stem-height                   1
stem-width                    0
stem-root               1838012
stem-surface            1321488
stem-color                   21
veil-type               1971545
veil-color              1826124
has-ring                     19
ring-type                 86195
spore-print-color       1899617
habitat                      25
season                        0
dtype: int64

In [66]:
test_new = test_df.drop(['id','stem-root','stem-surface','veil-type','veil-color','spore-print-color'],axis='columns')

In [67]:
test_new['cap-diameter'] = test_new['cap-diameter'].fillna(test_new['cap-diameter'].median())
test_new['stem-height'] = test_new['stem-height'].fillna(test_new['stem-height'].median())
test_new['stem-width'] = test_new['stem-width'].fillna(test_new['stem-width'].median())

In [68]:
test_new['cap-shape'] = test_new['cap-shape'].fillna(test_new['cap-shape'].value_counts().index[0])
test_new['cap-surface'] = test_new['cap-surface'].fillna(test_new['cap-surface'].value_counts().index[0])
test_new['cap-color'] = test_new['cap-color'].fillna(test_new['cap-color'].value_counts().index[0])
test_new['does-bruise-or-bleed'] = test_new['does-bruise-or-bleed'].fillna(test_new['does-bruise-or-bleed'].value_counts().index[0])
test_new['gill-attachment'] = test_new['gill-attachment'].fillna(test_new['gill-attachment'].value_counts().index[0])
test_new['gill-spacing'] = test_new['gill-spacing'].fillna(test_new['gill-spacing'].value_counts().index[0])
test_new['gill-color'] = test_new['gill-color'].fillna(test_new['gill-color'].value_counts().index[0])
test_new['stem-color'] = test_new['stem-color'].fillna(test_new['stem-color'].value_counts().index[0])
test_new['has-ring'] = test_new['has-ring'].fillna(test_new['has-ring'].value_counts().index[0])
test_new['ring-type'] = test_new['ring-type'].fillna(test_new['ring-type'].value_counts().index[0])
test_new['habitat'] = test_new['habitat'].fillna(test_new['habitat'].value_counts().index[0])
test_new['season'] = test_new['season'].fillna(test_new['season'].value_counts().index[0])

In [69]:
le = LabelEncoder()
test_new['cap-shape'] = le.fit_transform(test_new['cap-shape'])
test_new['cap-surface'] = le.fit_transform(test_new['cap-surface'])
test_new['cap-color'] = le.fit_transform(test_new['cap-color'])
test_new['does-bruise-or-bleed'] = le.fit_transform(test_new['does-bruise-or-bleed'])
test_new['gill-attachment'] = le.fit_transform(test_new['gill-attachment'])
test_new['gill-spacing'] = le.fit_transform(test_new['gill-spacing'])
test_new['gill-color'] = le.fit_transform(test_new['gill-color'])
test_new['stem-color'] = le.fit_transform(test_new['stem-color'])
test_new['has-ring'] = le.fit_transform(test_new['has-ring'])
test_new['ring-type'] = le.fit_transform(test_new['ring-type'])
test_new['habitat'] = le.fit_transform(test_new['habitat'])
test_new['season'] = le.fit_transform(test_new['season'])

In [70]:
test_new = pd.DataFrame(ss.fit_transform(test_new),columns=test_new.columns)

In [71]:
test_pred = ann.predict(test_new)

64937/64937 ━━━━━━━━━━━━━━━━━━━━ 67s 1ms/step


In [72]:
test_pred

array([[0.02527826],
       [0.9793284 ],
       [0.9896304 ],
       ...,
       [0.9928137 ],
       [0.02527826],
       [0.02527826]], dtype=float32)

In [73]:
prd_final = []
for i in test_pred:
    if i[0] > 0.5:
        prd_final.append(1)
    else:
        prd_final.append(0)

In [74]:
result = pd.DataFrame(prd_final)

In [75]:
result=result.rename(columns={0:'class'})

In [76]:
result['id'] = test_df['id']

In [77]:
result = result[['id','class']]

In [78]:
result['class'] = np.where(result['class']==0,'e','p')

In [79]:
result

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,e


In [80]:
result.to_csv('mushroom_sub5.csv',index=False)